In [ ]:
import os
import zipfile

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder1 = "/content/TFG/DeepXplore/GTSRB/Models_123/Resultats_DeepXplore_GTSRB"

def process_folder(folder):
    zip_files = [f for f in os.listdir(folder) if f.endswith('.zip')]
    txt_files = [f for f in os.listdir(folder) if f.endswith('.txt')]
    txt_paths = []

    for zip_file in zip_files:
        zip_path = os.path.join(folder, zip_file)
        extract_path = os.path.join('/content', zip_file.replace(".zip", ""))

        os.makedirs(extract_path, exist_ok=True)

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
            print(f"Extracted {zip_file} to {extract_path}")

    for txt_file in txt_files:
        txt_path = os.path.join(folder, txt_file)
        print(f"Found TXT file: {txt_path}")
        txt_paths.append(txt_path)

    return txt_paths


txt_deepxplore = process_folder(folder1)


Extracted generated_inputs_trafficsigns_occl0.zip to /content/generated_inputs_trafficsigns_occl0
Extracted generated_inputs_trafficsigns_occl2.zip to /content/generated_inputs_trafficsigns_occl2
Extracted generated_inputs_trafficsigns_occl1.zip to /content/generated_inputs_trafficsigns_occl1
Extracted generated_inputs_trafficsigns_light0.zip to /content/generated_inputs_trafficsigns_light0
Extracted generated_inputs_trafficsigns_light1.zip to /content/generated_inputs_trafficsigns_light1
Extracted generated_inputs_trafficsigns_light2.zip to /content/generated_inputs_trafficsigns_light2
Extracted generated_inputs_trafficsigns_blackout0.zip to /content/generated_inputs_trafficsigns_blackout0
Extracted generated_inputs_trafficsigns_blackout2.zip to /content/generated_inputs_trafficsigns_blackout2
Extracted generated_inputs_trafficsigns_blackout1.zip to /content/generated_inputs_trafficsigns_blackout1
Found TXT file: /content/drive/MyDrive/GIA/TFG/Senyals_deepXplore/Material/Bons_Models/R

In [ ]:
gtsrb_labels = {
    0: "Speed limit_20 km-h",
    1: "Speed limit_30 km-h",
    2: "Speed limit_50 km-h",
    3: "Speed limit_60 km-h",
    4: "Speed limit_70 km-h",
    5: "Speed limit_80 km-h",
    6: "End of speed limit_80 km-h",
    7: "Speed limit_100 km-h",
    8: "Speed limit_120 km-h",
    9: "No passing",
    10: "No passing for vehicles over 3.5 metric tons",
    11: "Right-of-way at the next intersection",
    12: "Priority road",
    13: "Yield",
    14: "Stop",
    15: "No vehicles",
    16: "Vehicles over 3.5 metric tons prohibited",
    17: "No entry",
    18: "General caution",
    19: "Dangerous curve to the left",
    20: "Dangerous curve to the right",
    21: "Double curve",
    22: "Bumpy road",
    23: "Slippery road",
    24: "Road narrows on the right",
    25: "Road work",
    26: "Traffic signals",
    27: "Pedestrians",
    28: "Children crossing",
    29: "Bicycles crossing",
    30: "Beware of ice-snow",
    31: "Wild animals crossing",
    32: "End of all speed and passing limits",
    33: "Turn right ahead",
    34: "Turn left ahead",
    35: "Ahead only",
    36: "Go straight or right",
    37: "Go straight or left",
    38: "Keep right",
    39: "Keep left",
    40: "Roundabout mandatory",
    41: "End of no passing",
    42: "End of no passing for vehicles over 3.5 metric tons"
}

def decode_label(pred):
    return gtsrb_labels[pred]

In [ ]:
import zipfile
with zipfile.ZipFile("/content/TFG/german_traffic_signs.zip", "r") as zip_ref:
    zip_ref.extractall("/content")

In [ ]:
import os
import re
import pandas as pd

def load_original_predictions(csv_file):
    """Loads original predictions from a CSV file into a dictionary using pandas."""
    df = pd.read_csv(csv_file)
    original_preds = dict(zip(df.iloc[:, 7], df.iloc[:, 6]))
    return original_preds

def process_predictions_deep(txt_files, original_preds):
    """Processes text files to compare model predictions with original predictions."""
    correct_preds = {}
    incorrect_preds = {}

    for file in txt_files:
        with open(file, "r") as f:
            data = f.read()
        lines = data.strip().split("\n")

        img_path = None
        txt_name = os.path.splitext(os.path.basename(file))[0][len("output_"):]
        predictions_after = ""

        for i, line in enumerate(lines):
            if line.startswith("Image path:"):
                img_path = line.split(":")[1].strip()

            elif line.startswith("Models predictions after:"):
                predictions_after = line

            elif img_path and line.startswith("./generated_inputs"):
                original_filename = os.path.basename(line.strip())
                modified_path = f"./generated_inputs_{txt_name}/{original_filename}"
                # dir_name = os.path.basename(os.path.dirname(modified_path))
                # result_num = int(dir_name.rsplit("_", 1)[-1][-1]) + 1

                prediction = "Unknown"
                if predictions_after:
                    pattern = re.compile(r"Model (\d+) -> ([^,]+) \(Conf: ([^\)]+)\), \(Original predicion conf: ([^\)]+)\)")
                    matches = pattern.findall(predictions_after)
                    incorrect_predictions = []
                    correct_predictions = []

                    for model_num, act_prediction, conf, original_conf in matches:
                        # print(act_prediction)
                        if img_path in original_preds:
                            original_class = decode_label(original_preds[img_path])
                            # print(original_class)
                            if act_prediction != original_class:
                                incorrect_predictions.append((modified_path, act_prediction))
                            else:
                                correct_predictions.append((modified_path, act_prediction))

                    if incorrect_predictions:
                        if img_path not in incorrect_preds:
                            incorrect_preds[img_path] = []
                        incorrect_preds[img_path].extend(incorrect_predictions)
                    if correct_predictions:
                        if img_path not in correct_preds:
                            correct_preds[img_path] = []
                        correct_preds[img_path].append(modified_path)

    return correct_preds, incorrect_preds

original_predictions = load_original_predictions("Test.csv")
correct_deep, incorrect_deep = process_predictions_deep(txt_deepxplore, original_predictions)

print("Correct Predictions:", correct_deep)
print("Incorrect Predictions:", incorrect_deep)


Correct Predictions: {'Test/02289.png': ['./generated_inputs_trafficsigns_occl0/occl_No entry_Yield_Yield_0.png', './generated_inputs_trafficsigns_light0/light_Keep right_Yield_Speed limit_20 km-h_0.png', './generated_inputs_trafficsigns_blackout2/blackout_Ahead only_Speed limit_30 km-h_Yield_0.png', './generated_inputs_trafficsigns_blackout0/blackout_Ahead only_Speed limit_30 km-h_Yield_0.png', './generated_inputs_trafficsigns_occl1/occl_Yield_Speed limit_30 km-h_Yield_0.png'], 'Test/06182.png': ['./generated_inputs_trafficsigns_occl0/occl_Yield_Speed limit_70 km-h_Speed limit_70 km-h_1.png', './generated_inputs_trafficsigns_occl2/occl_Speed limit_70 km-h_Speed limit_70 km-h_Road work_1.png', './generated_inputs_trafficsigns_occl1/occl_Speed limit_70 km-h_Yield_Speed limit_70 km-h_1.png'], 'Test/02962.png': ['./generated_inputs_trafficsigns_occl0/occl_Bumpy road_Yield_Yield_4.png', './generated_inputs_trafficsigns_occl1/occl_Bumpy road_Yield_Yield_4.png'], 'Test/02549.png': ['./genera

In [ ]:
from collections import defaultdict
merged = defaultdict(list)

for d in (incorrect_deep, correct_deep):
    for k, v in d.items():
        merged[k].append(v)

In [ ]:
import cv2
import numpy as np
img_wb = []
def is_white_or_black(image_path):
    image = cv2.imread(image_path)
    image = image.astype(np.float32)

    return np.all(image == 255) or np.all(image == 0)

filtered_predictions = []
for key, values in incorrect_deep.items():
    for img_path, prediction in values:
        if "light" in img_path:
            if is_white_or_black(img_path):
                filtered_predictions.append((img_path, prediction))

for img_path, pred in filtered_predictions:
    print(f"Image: {img_path} -> Prediction: {pred}")
    img_wb.append(img_path)



In [ ]:
len(img_wb)

891